In [ ]:
%matplotlib inline

In [ ]:
import sklearn
import sklearn.cluster as clustering
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
import numpy as np
import sklearn.metrics


In [ ]:
combined_asos = pd.read_csv("/share/share/combined_asos_2023.csv")

In [ ]:
combined_asos

In [ ]:
stations_lls = combined_asos.groupby("station")[["lat", "lon", "tmpf", "dwpf", "sknt"]].mean()

In [ ]:
conus_stations = stations_lls[np.logical_and(stations_lls['lat']<51, stations_lls['lat']>22) ]

In [ ]:
conus_stations

In [ ]:
plt.scatter(conus_stations['lon'], conus_stations['lat'])

In [ ]:
my_clustering = clustering.AgglomerativeClustering().fit(conus_stations)

In [ ]:
my_clustering.labels_

In [ ]:
plt.scatter(conus_stations['lon'], conus_stations['lat'], c=my_clustering.labels_)

In [ ]:
station_mean_vals = conus_stations.groupby('station')[['lat','lon','tmpf','dwpf', 'sknt']].mean()
station_mean_vals = station_mean_vals.dropna()

In [ ]:
def custom_distance(vec_1, vec_2):
    # should be in the order we select.
    # so here should be temperature, dewpoint, wind speed
    temp_dist = np.power(5*(vec_1[0]-vec_2[0]),2)
    dewp_dist = np.power(vec_1[1]-vec_2[1],2)
    sknt_dist = np.power(0.5*(vec_1[2]-vec_2[2]),2)
    return temp_dist + dewp_dist + sknt_dist

# it will be passed as a matrix, so we will use
def sklearn_custom_distance(X):
    return sklearn.metrics.pairwise_distances(X, metric=custom_distance)


In [ ]:
my_clustering = clustering.AgglomerativeClustering(metric=sklearn_custom_distance, n_clusters=5, linkage='average'
                                                   ).fit(station_mean_vals[['tmpf','dwpf', 'sknt']])

In [ ]:
plt.scatter(station_mean_vals['lon'], station_mean_vals['lat'], c=my_clustering.labels_)